In [1]:
import tensorflow as tf
import numpy as np
import pandas

/Users/olaf/.pyenv/versions/anaconda3/envs/tfp3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# 입력 데이터 준비

In [2]:
# LOAD TEXT DATA
dbpedia = tf.contrib.learn.datasets.load_dataset(
      'dbpedia', test_with_fake_data=False, size='large')

In [3]:
MAX_DOCUMENT_LENGTH = 1014
NUM_ALPHABET = 255 # ASCII SPACE
NUM_LABEL = 15
ASCII_SPACE = 32
NUM_SAMPLES = dbpedia.train.data.shape[0]
NUM_TESTSET = dbpedia.test.data.shape[0]

In [4]:
def concat_col(a, b):
    return a+b
v_concat_col = np.vectorize(concat_col)

# 데이터 순서를 섞기 위한 마스크 값.
# DBPEDIA 데이터가 LABEL 순으로 정렬되어있어서 미니배치마다 LABEL을 골고루 학습하기 위해 데이터의 순서를 섞어야 함.
mask = np.arange(NUM_SAMPLES)
np.random.shuffle(mask)

large_train_x = dbpedia.train.data[mask]
large_train_y = dbpedia.train.target[mask]

char_processor = tf.contrib.learn.preprocessing.ByteProcessor(MAX_DOCUMENT_LENGTH)

#x_test = np.char.upper(v_concat_col(dbpedia.test.data[:,0], dbpedia.test.data[:,1]))
#y_test = dbpedia.test.target
#x_test = np.array(list(char_processor.transform(x_test)))
#x_hot_test = np.zeros([NUM_SAMPLES, NUM_ALPHABET, MAX_DOCUMENT_LENGTH])
#for i in range(NUM_SAMPLES):
#    x_hot_test[i][x_test, np.arange(MAX_DOCUMENT_LENGTH)] = 1
#y_hot_test = np.zeros([NUM_SAMPLES, NUM_LABEL])
#y_hot_test[np.arange(NUM_TESTSET), y_test] = 1

# 모델 빌드

In [5]:
global_step = tf.Variable(0, trainable=False, name='global_step')

# X: NUM_ALPHABET X MAX_DOCUMENT_LENGTH (1014)

# data_format = the default format "NHWC", the data is stored in the order of: 
#    [batch, height, width, channels]
X = tf.placeholder(tf.float32, [None, 1, MAX_DOCUMENT_LENGTH, NUM_ALPHABET])
Y = tf.placeholder(tf.float32, [None, NUM_LABEL])

# filter: A Tensor. Must have the same type as input. A 4-D tensor of shape:
#    [filter_height, filter_width, in_channels, out_channels]
with tf.name_scope('layer1'):
    K1 = tf.Variable(tf.random_normal([1, 7, NUM_ALPHABET, 256], stddev=0.01))
    L1 = tf.nn.conv2d(X, K1, strides=[1,1,1,1], padding='VALID')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1,1,3,1], strides=[1,1,3,1], padding='VALID')
    print(L1) # L1: None, 1, 336, 256

Tensor("layer1/MaxPool:0", shape=(?, 1, 336, 256), dtype=float32)


In [6]:
with tf.name_scope('layer2'):
    K2 = tf.Variable(tf.random_normal([1, 7, 256, 256], stddev=0.01))
    L2 = tf.nn.conv2d(L1, K2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1,1,3,1], strides=[1,1,3,1], padding='VALID')
    print(L2) # L2: None, 1, 110, 256

Tensor("layer2/MaxPool:0", shape=(?, 1, 110, 256), dtype=float32)


In [7]:
with tf.name_scope('layer3'):
    K3 = tf.Variable(tf.random_normal([1, 3, 256, 256], stddev=0.01))
    L3 = tf.nn.conv2d(L2, K3, strides=[1,1,1,1], padding='VALID')
    L3 = tf.nn.relu(L3)
    print(L3) # L3: None, 1, 108, 256

Tensor("layer3/Relu:0", shape=(?, 1, 108, 256), dtype=float32)


In [8]:
with tf.name_scope('layer4'):
    K4 = tf.Variable(tf.random_normal([1, 3, 256, 256], stddev=0.01))
    L4 = tf.nn.conv2d(L3, K4, strides=[1,1,1,1], padding='VALID')
    L4 = tf.nn.relu(L4)
    print(L4) # L4: None, 1, 106, 256

Tensor("layer4/Relu:0", shape=(?, 1, 106, 256), dtype=float32)


In [9]:
with tf.name_scope('layer5'):
    K5 = tf.Variable(tf.random_normal([1, 3, 256, 256], stddev=0.01))
    L5 = tf.nn.conv2d(L4, K5, strides=[1,1,1,1], padding='VALID')
    L5 = tf.nn.relu(L5)
    print(L5) # L5: None, 1, 104, 256

Tensor("layer5/Relu:0", shape=(?, 1, 104, 256), dtype=float32)


In [10]:
with tf.name_scope('layer6'):
    K6 = tf.Variable(tf.random_normal([1, 3, 256, 256], stddev=0.01))
    L6 = tf.nn.conv2d(L5, K6, strides=[1,1,1,1], padding='VALID')
    L6 = tf.nn.relu(L6)
    L6 = tf.nn.max_pool(L6, ksize=[1,1,3,1], strides=[1,1,3,1], padding='VALID')
    print(L6) # L6: None, 1, 34, 256

Tensor("layer6/MaxPool:0", shape=(?, 1, 34, 256), dtype=float32)


In [11]:
with tf.name_scope('layer7'):
    W7 = tf.Variable(tf.random_normal([1*34*256, 1024], stddev=0.01))
    L7 = tf.reshape(L6, [-1, 1*34*256])
    L7 = tf.matmul(L7, W7)
    L7 = tf.nn.relu(L7)
    L7 = tf.nn.dropout(L7, 0.8)
    print(L7) # L7: 1, 1024

with tf.name_scope('layer8'):
    W8 = tf.Variable(tf.random_normal([1024, 1024], stddev=0.1))
    L8 = tf.matmul(L7, W8)
    L8 = tf.nn.relu(L8)
    L8 = tf.nn.dropout(L8, 0.8)
    print(L8) # L8: 1, 1024

with tf.name_scope('output'):
    W9 = tf.Variable(tf.random_normal([1024, NUM_LABEL], stddev=0.01))
    model = tf.matmul(L8, W9)
    print(model) # L9: 1, 2

Tensor("layer7/dropout/mul:0", shape=(?, 1024), dtype=float32)
Tensor("layer8/dropout/mul:0", shape=(?, 1024), dtype=float32)
Tensor("output/MatMul:0", shape=(?, 15), dtype=float32)


In [12]:
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
    training = tf.train.AdamOptimizer(0.01).minimize(cost, global_step=global_step)
    tf.summary.scalar('cost', cost)
    tf.summary.histogram("Weights1", K1)
    W1 = tf.reshape(K1, [-1, NUM_ALPHABET, 7, 1])
    tf.summary.image("Filter1", W1, max_outputs=3)

# 모델 학습

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('logs', sess.graph)

batch_size = 10
total_batch = int(large_train_x.shape[0] / batch_size)

print("total batch: %d" % total_batch)

for epoch in range(2):
    for b in range(total_batch):
        minibatch_x = large_train_x[b*batch_size : (b+1)*batch_size]
        minibatch_y = large_train_y[b*batch_size : (b+1)*batch_size]
        
        # 알파벳은 모두 대문자로 치환 / 제목+본문을 합하여 문장을 만든다.
        minibatch_x = np.char.upper(v_concat_col(minibatch_x[:,0], minibatch_x[:,1]))
#        print(minibatch_x[0])
#        print(minibatch_y[0])

        # 알파벳을 ASCII DEC 값으로 변환한다.
        minibatch_x = np.array(list(char_processor.fit_transform(minibatch_x)))
#        print(minibatch_x[0])
#        print(minibatch_y[0])

        # ONE-HOT-ENCODING
        x_hot_train = np.zeros([batch_size, MAX_DOCUMENT_LENGTH, NUM_ALPHABET])
        for i in range(batch_size):
            x_hot_train[i][np.arange(MAX_DOCUMENT_LENGTH), minibatch_x] = 1
        x_hot_train = x_hot_train.reshape(-1, 1, MAX_DOCUMENT_LENGTH, NUM_ALPHABET)
#        print(x_hot_train.shape)
        
        y_hot_train = np.zeros([batch_size, NUM_LABEL])
        y_hot_train[np.arange(batch_size), minibatch_y] = 1
#        print(x_hot_train[0])
#        print(y_hot_train[0])
        
        # 학습 - cost optimization
        _, cost_val, output = sess.run([training, cost, model], feed_dict={X:x_hot_train, Y:y_hot_train})
        
        # 정확도 계산
        prediction = np.argmax(output, 1)
        accuracy = np.mean(prediction == minibatch_y)
        print("Epoch:%02d, Mini Batch: %04d, Cost: %.3f, Accuracy: %.2f" % (epoch+1, b, cost_val, accuracy))
        
        summary = sess.run(merged, feed_dict={X:x_hot_train, Y:y_hot_train})
        writer.add_summary(summary, global_step=sess.run(global_step))

total batch: 56000
Epoch:01, Mini Batch: 0000, Cost: 2.708, Accuracy: 0.10
Epoch:01, Mini Batch: 0001, Cost: 49.501, Accuracy: 0.20
Epoch:01, Mini Batch: 0002, Cost: 2.707, Accuracy: 0.00
Epoch:01, Mini Batch: 0003, Cost: 3.033, Accuracy: 0.10
Epoch:01, Mini Batch: 0004, Cost: 2.705, Accuracy: 0.00
Epoch:01, Mini Batch: 0005, Cost: 2.684, Accuracy: 0.20
Epoch:01, Mini Batch: 0006, Cost: 52.288, Accuracy: 0.10
Epoch:01, Mini Batch: 0007, Cost: 2.897, Accuracy: 0.10
Epoch:01, Mini Batch: 0008, Cost: 2.698, Accuracy: 0.10
Epoch:01, Mini Batch: 0009, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0010, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0011, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0012, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0013, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0014, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0015, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0016, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0017,

Epoch:01, Mini Batch: 0146, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0147, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0148, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0149, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0150, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0151, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0152, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0153, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0154, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0155, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0156, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0157, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0158, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0159, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0160, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0161, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0162, Cost: 2.708, Accuracy: 0.00
Epoch:01, Mini Batch: 0163, Cost: 2.708, Accurac

KeyboardInterrupt: 

In [24]:
data = [[[[1,2,3],[4,5,6]]]]
data_np = np.asarray(data, np.float32)
print(data_np.shape)

data_tf = tf.convert_to_tensor(data_np, np.float32)
print(data_tf)

sxs = tf.InteractiveSession()  
print(data_tf.eval())
sxs.close()

(1, 1, 2, 3)
Tensor("Const_5:0", shape=(1, 1, 2, 3), dtype=float32)
[[[[ 1.  2.  3.]
   [ 4.  5.  6.]]]]
